In [ ]:
import os
import sys

ENDSWITH = 'OCR'

NOTEBOOK_DIR = os.getcwd()

if not NOTEBOOK_DIR.endswith(ENDSWITH):
    raise ValueError(f"Not in correct dir, expect end with {ENDSWITH}, but got {NOTEBOOK_DIR} instead")

BASE_DIR = os.path.abspath(os.path.join(NOTEBOOK_DIR, '..', '..', '..', '..'))
print(BASE_DIR)

sys.path.insert(0, os.path.join(BASE_DIR, 'code'))

In [ ]:
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")


In [ ]:
from MangaOCREvaluator import ParseAnnotation, MangaOCREvaluator
from pipeline.OCRModels.MangaOCRModel import MangaOCRModel

In [ ]:
# Parse XML to JSON (one-time setup)
manga_name = "AisazuNihaIrarenai"
xml_path = os.path.join(BASE_DIR, 'data', 'Manga109_released_2023_12_07', 'annotations', f'{manga_name}.xml')
images_dir = os.path.join(BASE_DIR, 'data', 'Manga109_released_2023_12_07', 'images', manga_name)
output_dir = os.path.join(BASE_DIR, 'data', 'MangaOCR', 'jsons_processed')

parser = ParseAnnotation(xml_path, images_dir, output_dir)
json_output_path = parser.parse_and_save()

In [ ]:
evaluator = MangaOCREvaluator(device=device)

ocr_model = MangaOCRModel()

metrics = evaluator.compare_bbox_types(
    ocr_model=ocr_model,
    json_path=str(json_output_path),
    images_dir=images_dir,
    batch_size=1,
    verbose=False,
    max_images=5
)